In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [3]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

In [7]:
print(iris['data'].shape, iris['feature_names'], iris['target'].shape, iris['target_names'])

(150, 4) ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] (150,) ['setosa' 'versicolor' 'virginica']


In [11]:
iris['target'][:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

labels are sorted

In [12]:
np.random.seed(10)
rnd_indices = np.random.permutation(iris['target'].shape[0])

In [18]:
# shuffle the X and y, and intercept to X
X = iris['data'][rnd_indices]
y = iris['target'][rnd_indices]
X_b = np.c_[np.ones([len(X),1]), X]

In [23]:
# train, val, test split
X_train, X_val, X_test = X_b[:90], X_b[90:120], X_b[120:]
y_train, y_val, y_test = y[:90], y[90:120], y[120:]

#### categoritize labels

In [50]:
from sklearn.preprocessing import OneHotEncoder
OneHot = OneHotEncoder(sparse=False)
y_train_c = OneHot.fit_transform(y_train.reshape(-1,1))
y_val_c = OneHot.transform(y_val.reshape(-1,1))
y_test_c = OneHot.transform(y_test.reshape(-1,1))

softmax score for class k:

$s_k(\mathbf{x})=\mathbf{x}^T\mathbf{\theta}^{(k)}$

softmax function:

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

So the equations we will need are the cost function:

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

And the equation for the gradients:

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

Note that $\log\left(\hat{p}_k^{(i)}\right)$ may not be computable if $\hat{p}_k^{(i)} = 0$. So we will add a tiny value $\epsilon$ to $\log\left(\hat{p}_k^{(i)}\right)$ to avoid getting `nan` values.

In [102]:
eta = 0.01           # learning rate
m = len(X_train)     # instance rows
epsilon = 10**(-7)  # small positive number to avoid nan
n_iterations = 5001
theta = np.random.randn(X_train.shape[1], y_train_c.shape[1]) # random initialization, theta is alwaays diagonal against X, (5,3)

def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

for iteration in range(n_iterations):
    score = X_train.dot(theta)   # probably of each class (90,5)*(5,3)
    y_prob = softmax(score)      # (90,3)
    loss = -np.mean(np.sum(y_train_c * np.log(y_prob + epsilon), axis=1))   # loss function for logistic regression is still MSE
    gradients = 1/m * X_train.T.dot(y_prob - y_train_c)  # (5,90)*(90,3)
    theta = theta - eta*gradients
    if iteration % 500 == 0:
        print(iteration, loss)

0 5.116997060758296
500 0.42627276322546365
1000 0.3514195911116283
1500 0.3067304355407432
2000 0.27518666437267236
2500 0.25147669365325015
3000 0.23295057657761428
3500 0.21805514741926596
4000 0.2058054253503143
4500 0.19554408165849232
5000 0.1868152809764123


In [103]:
# model parameter after 5001 iternations
theta

array([[ 0.72753952,  0.09550867, -2.07416014],
       [-0.15804361,  0.78746446, -0.43524657],
       [ 3.21150709,  0.11200907, -1.38785118],
       [-1.55811271,  0.36487996,  2.22211255],
       [-1.39377266, -1.77050988,  1.1902918 ]])

In [104]:
# score validation set
score = X_val.dot(theta)
y_prob = softmax(score) # (30*3) matrix of estimated probabilities
y_predict = np.argmax(y_prob, axis=1)

accuracy_score = np.mean(y_predict == y_val)
accuracy_score

1.0

#### Add early stopping

In [167]:
eta = 0.05           # learning rate
m = len(X_train)     # instance rows
epsilon = 10**(-7)  # small positive number to avoid nan
n_iterations = 5001
theta = np.random.randn(X_train.shape[1], y_train_c.shape[1]) # random initialization, theta is alwaays diagonal against X, (5,3)

alpha = 0.5  # regularization hyperparameter
best_loss = np.infty
threshold, consecutive = 100, 0

for iteration in range(n_iterations):
    score = X_train.dot(theta)   # probably of each class (90,5)*(5,3)
    y_prob = softmax(score)      # (90,3)
    xentropy_loss = -np.mean(np.sum(y_train_c * np.log(y_prob + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(theta[1:])) # no need to add loss on bias (intercept)
    loss = xentropy_loss + alpha * l2_loss
    gradients = 1/m * X_train.T.dot(y_prob - y_train_c)  # (5,90)*(90,3)
    theta = theta - eta*gradients
    
    score = X_val.dot(theta)   # probably of each class (90,5)*(5,3)
    y_prob = softmax(score)      # (90,3)
    xentropy_loss = -np.mean(np.sum(y_val_c * np.log(y_prob + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(theta[1:])) # use the theta from training
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 10 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
        consecutive = 0
    else:
        consecutive += 1
    if consecutive >= threshold:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break

0 9.293419031437885
10 1.9247294285507413
20 1.9240660895486263
30 1.9382071937700376
40 1.9602582682608074
50 1.9869232579823382
60 2.016469219393534
70 2.047931385803968
80 2.0807478403863757
90 2.1145789434351903
100 2.1492126568655445
110 2.1845124718562237
114 1.9218418481984794
115 2.2023825566029998 early stopping!


In [168]:
# score validation set
score = X_val.dot(theta)
y_prob = softmax(score) # (30*3) matrix of estimated probabilities
y_predict = np.argmax(y_prob, axis=1)

accuracy_score = np.mean(y_predict == y_val)
accuracy_score

0.9666666666666667

In [169]:
logits = X_test.dot(theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9